<h1>Version 1</h1>
<p>This is simplest version with just a heater for warmth and intake fan to lower humidity</p>

<h3>Pseudo code</h3>
<pre>
humiditySetPoint
humidHigh = humiditySetPoint + something
humidLow = humiditySetPoint - something

tempSetPoint
tempHigh = tempSetPoint + something
tempLow = tempSetPoint - something

main loop:
   read temp and humidity
   if humidity > humidHigh
      if intake fan off
         turn intake fan on
   else if humidity < humidLow
      if intake fan on
         turn intake fan off
         
   if temp > tempHigh
      if heat on
         turn heat off
   else if temp < tempLow
      if heat off
         turn heat on
         
   display temp
   display humidity
   display heater on/off
   display fan on/off
   
   check buttons down
   if ok button
      switch selected field
         temp:
         delta temp:
         humidity:
         delta humidity:
   update menu display()
</pre>

In [ ]:
%run -i 'sausage.py'

In [1]:
#Imports

import time

#https://www.instructables.com/Raspberry-Pi-Tutorial-How-to-Use-the-DHT-22/
#git clone https://github.com/adafruit/Adafruit_Python_DHT.git
import Adafruit_DHT as dht

#https://randomnerdtutorials.com/raspberry-pi-digital-inputs-python/
#https://miketrebilcock.github.io/js-gpiozero/DigitalOutputDevice.html  ?
from gpiozero import DigitalOutputDevice

#https://www.waveshare.com/wiki/1.3inch_OLED_HAT
import SH1106
import config
import traceback
from PIL import Image,ImageDraw,ImageFont

#Display setup and variables
try:
    display = SH1106.SH1106()
    display.Init()
    display.clear()
except IOError as e:
    print(e)
font = ImageFont.truetype('Font.ttf',11)

#Threading
import multiprocessing

In [2]:
#Setup and Globals

#Humidity and Temperature
humidTarget = 10
humidDelta = 5
fanOn = False
tempTarget = 75
tempDelta = 2
heatOn = False

#GPIO
fanPin = 14
heaterPin = 15
humidPin = 23
fanOutput = DigitalOutputDevice(fanPin)
heaterOutput = DigitalOutputDevice(heaterPin)

#Other globals
sensorLastRead = 0
humidTemp = (0, 0)
ignoreButton = False

#Temp Target          Temp Delta
#Humid Target         Humid Delta
menuCol = 0
menuRow = 0

In [3]:
#Functions

def ReadHumidTemp(humid_pin, humid_temp_pipe, log_pipe):
    hp = humid_pin
    try:
        while True:
            time.sleep(5)
            print("Reading pin %d" % humidPin)
            #retries=15, delay_seconds=2,
            h,t = dht.read_retry(dht.DHT22, hp)#, retries=0, delay_seconds=2)
            if t is None:
                print("Read Failed.")
                continue
            t = (t * 9 / 5) + 32
            humid_temp_pipe.send((h,t))
            logString = 'Temp={0:0.1f}*F  Humidity={1:0.1f}%'.format(t,h)
            log_pipe.send(logString)
    except KeyboardInterrupt:
        print("HT_KeyboardInterrupt")
        
def UpdateDisplay():
    global menuRow
    global menuCol
    global humidTarget
    global tempTarget
    global humidDelta
    global tempDelta
    global humidTemp
    #Create all the strings first
    if humidTemp is None:
        humidTempStatus = "Humidity:%d%%  Temp:%dF" % (0,0)
    else:
        humidTempStatus = "Humidity:%d%%  Temp:%dF" % humidTemp
    fanStatus = "ON" if fanOn else "OFF"
    fanStatus = "Fan:" + fanStatus
    heatStatus = "ON" if heatOn else "OFF"
    heatStatus = "Heater:" + heatStatus
    heatFanStatus = heatStatus + "  " + fanStatus
    #Target strings
    tempTargetStatus = "Target[F]:%d" % tempTarget
    tempDeltaStatus = "Δ[F]:%d" % tempDelta
    humidTargetStatus = "Target[%%]:%d" % humidTarget
    humidDeltaStatus = "Δ[%%]:%d" % humidDelta
    #Prepend a '-' to whatever is selected
    if menuCol == 0:
        if menuRow == 0:
            tempTargetStatus = "-" + tempTargetStatus
            tempDeltaStatus = " " + tempDeltaStatus
            humidTargetStatus = " " + humidTargetStatus
            humidDeltaStatus = " " + humidDeltaStatus
        else:
            tempTargetStatus = " " + tempTargetStatus
            tempDeltaStatus = " " + tempDeltaStatus
            humidTargetStatus = "-" + humidTargetStatus
            humidDeltaStatus = " " + humidDeltaStatus
    else:
        if menuRow == 0:
            tempTargetStatus = " " + tempTargetStatus
            tempDeltaStatus = "-" + tempDeltaStatus
            humidTargetStatus = " " + humidTargetStatus
            humidDeltaStatus = " " + humidDeltaStatus
        else:
            tempTargetStatus = " " + tempTargetStatus
            tempDeltaStatus = " " + tempDeltaStatus
            humidTargetStatus = " " + humidTargetStatus
            humidDeltaStatus = "-" + humidDeltaStatus
    
    #Create image that we'll use
    imageDisplay = Image.new('1', (display.width, display.height), "WHITE")
    draw = ImageDraw.Draw(imageDisplay)
    
    #Draw the strings to the image
    draw.text((0,0), humidTempStatus, font=font, fill=0)
    draw.text((0,15), heatFanStatus, font=font, fill=0)
    draw.text((0,30), tempTargetStatus + "  " + tempDeltaStatus, font=font, fill=0)
    draw.text((0,45), humidTargetStatus + "  " + humidDeltaStatus, font=font, fill=0)
    
    #Draw the image to the display
    display.ShowImage(display.getbuffer(imageDisplay))
    
def ReadButtons():
    global ignoreButton
    global menuRow
    global menuCol
    global humidTarget
    global tempTarget
    global humidDelta
    global tempDelta
    buttonUp = True if display.RPI.digital_read(display.RPI.GPIO_KEY_UP_PIN ) != 0 else False
    buttonDown = True if display.RPI.digital_read(display.RPI.GPIO_KEY_DOWN_PIN ) != 0 == 0 else False
    buttonLeft = True if display.RPI.digital_read(display.RPI.GPIO_KEY_LEFT_PIN ) != 0 else False
    buttonRight = True if display.RPI.digital_read(display.RPI.GPIO_KEY_RIGHT_PIN ) != 0 else False
    buttonIncrease = True if display.RPI.digital_read(display.RPI.GPIO_KEY1_PIN ) != 0 == 0 else False
    buttonDecrease = True if display.RPI.digital_read(display.RPI.GPIO_KEY3_PIN ) != 0 else False
    
    #If button has been pressed, then ignore it until it's released
    if buttonUp is False and buttonDown is False and buttonLeft is False and buttonRight is False and buttonIncrease is False and buttonDecrease is False:
        if ignoreButton is True:
            print("Reset")
            ignoreButton = False
        return False
    
    #Figure out which button is pressed: Either move menu selection or increase/decrease value
    ignoreButton = True
    changeValue = 0
    if buttonUp is True:
        print("ButtonUp")
        if menuRow == 1:
            menuRow = 0
        return True
    elif buttonDown is True:
        print("ButtonDown")
        if menuRow == 0:
            menuRow = 1
        return True
    elif buttonLeft is True:
        print("ButtonLeft")
        if menuCol == 1:
            menuCol = 0
        return True
    elif buttonRight is True:
        print("ButtonRight")
        if menuCol == 0:
            menuCol = 1
        return True
    elif buttonIncrease is True:
        print("ButtonIncrease")
        changeValue = 1
    elif buttonDecrease is True:
        print("ButtonDecrease")
        changeValue = -1

    #If we get this far, all that's left is possible increase/decrease
    if changeValue == 0:
        #No buttons are pressed
        return False
    if menuCol == 0:
        if menuRow == 0:
            tempTarget += changeValue
        else:
            humidTarget += changeValue
    else:
        if menuRow == 0:
            tempDelta += changeValue
            if tempDelta < 1:
                tempDelta = 1
        else:
            humidDelta += changeValue
            if humidDelta < 1:
                humidDelta = 1
    return True

In [ ]:
#Main Loop
humidTempParent,humidTempChild = multiprocessing.Pipe()
logParent,logChild = multiprocessing.Pipe()
processDHT = multiprocessing.Process(target=ReadHumidTemp, name='thrDHT', args=(humidPin,humidTempChild,logChild))
processDHT.start()
updateDisplayNow = True
try:
    while True:
        time.sleep(0.05)
        #dht takes a few to read, so wait 10 sec before checking
        now = time.time()
        if now - sensorLastRead > 5:
            #Read the sensor
            #humidTemp = ReadHumidTemp()
            sensorLastRead = time.time()
            if humidTempParent.poll() is False:
                print("No pipe yet")
                continue
            print("hit the pipe")
            humidTemp = humidTempParent.recv()
            logString = logParent.recv()
            print(logString)
            updateDisplayNow = True

            #Turn Fan on/off
            humidHigh = humidTarget + humidDelta
            humidLow = humidTarget - humidDelta
            if humidTemp[0] > humidHigh:
                if fanOn is False:
                    #turn intake fan on
                    fanOutput.on()
                    fanOn = True
                    print('on')
            elif humidTemp[0] < humidLow:
                if fanOn is True:
                    #turn intake fan off
                    fanOutput.off()
                    fanOn = False
                    print('off')#GPIO.output(fanPin, False)
            #Turn Heater on/off
            tempHigh = tempTarget + tempDelta
            tempLow = tempTarget - tempDelta
            if humidTemp[1] > tempHigh:
                if heatOn is True:
                    #turn heat off
                    heaterOutput.off()
                    heatOn = False
                    print('off')#GPIO.output(heaterPin, False)
            elif humidTemp[1] < tempLow:
                if heatOn is False:
                    #turn heat on
                    heaterOutput.on()
                    heatOn = True
                    print('on')# GPIO.output(heaterPin, True)

        #Read Buttons, handle menu stuff
        if ReadButtons() is True or updateDisplayNow is True:
            #Now update the display
            UpdateDisplay()
            updateDisplayNow = False
except KeyboardInterrupt:
    print("KeyboardInterrupt")
processDHT.join()

No pipe yet
Reading pin 23
No pipe yet
hit the pipe
Temp=69.1*F  Humidity=13.6%
on
Reading pin 23
hit the pipe
Temp=69.1*F  Humidity=13.6%
Reading pin 23
hit the pipe
Temp=69.1*F  Humidity=13.5%
Reading pin 23
hit the pipe
Temp=69.1*F  Humidity=13.5%
Reading pin 23
No pipe yet
Reading pin 23
hit the pipe
Temp=69.1*F  Humidity=13.6%
hit the pipe
Temp=68.9*F  Humidity=13.6%
Reading pin 23


<h2>Below this is alternate menu</h2>

In [ ]:
def UpdateDisplay():
    DisplayStatus()
    
def DisplayStatus():
    #Create all the strings first
    humidTempStatus = "   %dF            %d%%" % humidTemp
    fanStatus = "ON" if fanOn else "OFF"
    fanStatus = "Fan:" + fanStatus
    heatStatus = "ON" if heatOn else "OFF"
    heatStatus = "Heat:" + heatStatus
    heatFanStatus = heatStatus + "  " + fanStatus
    #Target strings
    tempTargetStatus = "Target[F]:%d" % tempTarget
    humidTargetStatus = "Target[%%]:%d" % humidTarget
    
    #Create image that we'll use
    imageDisplay = Image.new('1', (display.width, display.height), "WHITE")
    draw = ImageDraw.Draw(imageDisplay)
    
    #Draw the strings to the image
    draw.text((0,0), humidTempStatus, font=font, fill=0)
    draw.text((0,15), heatFanStatus, font=font, fill=0)
    draw.text((0,30), tempTargetStatus, font=font, fill=0)
    draw.text((0,45), humidTargetStatus, font=font, fill=0)
    
    #Draw the image to the display
    display.ShowImage(display.getbuffer(imageDisplay))

<h2>Below this is testing code</h2>

<h3>Write image to a picture<h/3>

In [ ]:
fanOn = False
heatOn = False
humidTemp = (85,50)
tempTarget = 85
humidTarget = 50
tempDelta = 2
humidDelta = 2
menuCol = 0
menuRow = 0
class obj:
    def __init__(self):
        self.width = 128
        self.height = 64
    def ShowImage(self, image):
        image.save("test.bmp")
    def getbuffer(self, image):
        return image
display = obj()
font = ImageFont.truetype('Font.ttf',11)

UpdateDisplay()

<h3>Write image to screen</h3>

In [ ]:
fanOn = False
heatOn = False
humidTemp = (85,50)
tempTarget = 85
humidTarget = 50
tempDelta = 2
humidDelta = 2
menuCol = 0
menuRow = 0

font = ImageFont.truetype('Font.ttf',11)

UpdateDisplay()

<h3>Button Test</h3>

In [ ]:
import SH1106
import time
from gpiozero import *

disp = SH1106.SH1106()
disp.Init()

In [ ]:
ignore = False
while True:
    # with canvas(device) as draw:
    if disp.RPI.digital_read(disp.RPI.GPIO_KEY_UP_PIN ) != 0:
        if ignore is False:
            print("ButtonUp Pressed")
            ignore = True
    elif ignore is True:
        print("Button Released")
        ignore = False

<h1>Version 2</h1>
<p>This version will have both heater, and thermoelectric cooler. Still simple menu design</p>

In [ ]:
#Setup and Globals

#Humidity and Temperature
humidTarget = 10
humidDelta = 5
fanOn = False
tempTarget = 75
tempDelta = 2
heatOn = False
coolerOn = False
dehumidifyOn = False
humidifyOn = False

#GPIO
dehumidifyPin = -1
humidifyPin = -1
fanPin = 14
heaterPin = 15
coolerPin = 18
humidPin = 23

#Other globals
sensorLastRead = 0
humidTemp = (0, 0)
ignoreButton = False

#Temp Target          Temp Delta
#Humid Target         Humid Delta
menuCol = 0
menuRow = 0

In [ ]:
#Functions

def ReadHumidTemp(humid_pin, humid_temp_pipe):
    hp = humid_pin
    try:
        while True:
            time.sleep(5)
            print("Reading pin %d" % humidPin)
            #retries=15, delay_seconds=2,
            h,t = dht.read_retry(dht.DHT22, hp)#, retries=0, delay_seconds=2)
            if t is None:
                print("Read Failed.")
                continue
            t = (t * 9 / 5) + 32
            print('Temp={0:0.1f}*F  Humidity={1:0.1f}%'.format(t,h))
            humid_temp_pipe.send((h,t))
    except KeyboardInterrupt:
        print("HT_KeyboardInterrupt")
        
def UpdateDisplay():
    #Create all the strings first
    humidTempStatus = "Temp:%dF  Humidity:%d%%" % humidTemp
    fanStatus = "•" if fanOn else " "
    fanStatus = "Fan:" + fanStatus
    heatStatus = "•" if heatOn else " "
    heatStatus = "Heat:" + heatStatus
    coolerStatus = "•" if coolerOn else " "
    coolerStatus = "Cool:" + coolerStatus
    heatFanStatus = heatStatus + "  " + coolerStatus + "  " + fanStatus
    #Target strings
    tempTargetStatus = "Target[F]:%d" % tempTarget
    tempDeltaStatus = "Δ[F]:%d" % tempDelta
    humidTargetStatus = "Target[%%]:%d" % humidTarget
    humidDeltaStatus = "Δ[%%]:%d" % humidDelta
    #Prepend a '-' to whatever is selected
    if menuCol == 0:
        if menuRow == 0:
            tempTargetStatus = "-" + tempTargetStatus
            tempDeltaStatus = " " + tempDeltaStatus
            humidTargetStatus = " " + humidTargetStatus
            humidDeltaStatus = " " + humidDeltaStatus
        else:
            tempTargetStatus = " " + tempTargetStatus
            tempDeltaStatus = " " + tempDeltaStatus
            humidTargetStatus = "-" + humidTargetStatus
            humidDeltaStatus = " " + humidDeltaStatus
    else:
        if menuRow == 0:
            tempTargetStatus = " " + tempTargetStatus
            tempDeltaStatus = "-" + tempDeltaStatus
            humidTargetStatus = " " + humidTargetStatus
            humidDeltaStatus = " " + humidDeltaStatus
        else:
            tempTargetStatus = " " + tempTargetStatus
            tempDeltaStatus = " " + tempDeltaStatus
            humidTargetStatus = " " + humidTargetStatus
            humidDeltaStatus = "-" + humidDeltaStatus
    
    #Create image that we'll use
    imageDisplay = Image.new('1', (display.width, display.height), "WHITE")
    draw = ImageDraw.Draw(imageDisplay)
    
    #Draw the strings to the image
    draw.text((0,0), humidTempStatus, font=font, fill=0)
    draw.text((0,15), heatFanStatus, font=font, fill=0)
    draw.text((0,30), tempTargetStatus + "  " + tempDeltaStatus, font=font, fill=0)
    draw.text((0,45), humidTargetStatus + "  " + humidDeltaStatus, font=font, fill=0)
    
    #Draw the image to the display
    display.ShowImage(display.getbuffer(imageDisplay))
    
def ReadButtons():
    global ignoreButton
    global menuRow
    global menuCol
    global humidTarget
    global tempTarget
    global humidDelta
    global tempDelta
    buttonUp = True if display.RPI.digital_read(display.RPI.GPIO_KEY_UP_PIN ) != 0 else False
    buttonDown = True if display.RPI.digital_read(display.RPI.GPIO_KEY_DOWN_PIN ) != 0 == 0 else False
    buttonLeft = True if display.RPI.digital_read(display.RPI.GPIO_KEY_LEFT_PIN ) != 0 else False
    buttonRight = True if display.RPI.digital_read(display.RPI.GPIO_KEY_RIGHT_PIN ) != 0 else False
    buttonIncrease = True if display.RPI.digital_read(display.RPI.GPIO_KEY1_PIN ) != 0 == 0 else False
    buttonDecrease = True if display.RPI.digital_read(display.RPI.GPIO_KEY3_PIN ) != 0 else False
    
    #If button has been pressed, then ignore it until it's released
    if buttonUp is False and buttonDown is False and buttonLeft is False and buttonRight is False and buttonIncrease is False and buttonDecrease is False:
        if ignoreButton is True:
            print("Reset")
            ignoreButton = False
        return
    
    #Figure out which button is pressed: Either move menu selection or increase/decrease value
    ignoreButton = True
    changeValue = 0
    if buttonUp is True:
        print("ButtonUp")
        if menuRow == 1:
            menuRow = 0
        return
    elif buttonDown is True:
        print("ButtonDown")
        if menuRow == 0:
            menuRow = 1
        return
    elif buttonLeft is True:
        print("ButtonLeft")
        if menuCol == 1:
            menuCol = 0
        return
    elif buttonRight is True:
        print("ButtonRight")
        if menuCol == 0:
            menuCol = 1
        return
    elif buttonIncrease is True:
        print("ButtonIncrease")
        changeValue = 1
    elif buttonDecrease is True:
        print("ButtonDecrease")
        changeValue = -1

    #If we get this far, all that's left is possible increase/decrease
    if changeValue == 0:
        #No buttons are pressed
        return
    if menuCol == 0:
        if menuRow == 0:
            tempTarget += changeValue
        else:
            humidTarget += changeValue
    else:
        if menuRow == 0:
            tempDelta += changeValue
            if tempDelta < 1:
                tempDelta = 1
        else:
            humidDelta += changeValue
            if humidDelta < 1:
                humidDelta = 1
    

In [ ]:
#dehumidifyPin = -1
#humidifyPin = -1
#fanPin = 14
#heaterPin = 15
#coolerPin = 18
fanRaiseHumid = False
fanLowerHumid = True
dehumidifyPresent = False
humidifyPresent = False
fanOutput = DigitalOutputDevice(fanPin)
heaterOutput = DigitalOutputDevice(heaterPin)
coolerOutput = DigitalOutputDevice(coolerPin)
humidifyOutput = None#DigitalOutputDevice(humidifyPin)
dehumidifyOutput = None#DigitalOutputDevice(dehumidifyPin)
#Main Loop
humidTempParent,humidTempChild = multiprocessing.Pipe()
processDHT = multiprocessing.Process(target=ReadHumidTemp, name='thrDHT', args=(humidPin,humidTempChild))
processDHT.start()
try:
    while True:
        time.sleep(0.5)

        #dht takes a few to read, so wait 10 sec before checking
        now = time.time()
        if now - sensorLastRead > 10:
            #Read the sensor
            #humidTemp = ReadHumidTemp()
            sensorLastRead = time.time()
            if humidTempParent.poll() is False:
                print("No pipe yet")
                continue
            print("hit the pipe")
            humidTemp = humidTempParent.recv()

            #Control Humidity
            humidHigh = humidTarget + humidDelta
            humidLow = humidTarget - humidDelta
            if humidTemp[0] > humidTarget and (fanOn is True or humidifyOn is True):
                #It was too dry and the humidity is back up to target
                if fanOn is True:
                    fanOutput.off()
                    fanOn = False
                    print('fan off')
                elif humidifyOn is True:
                    humidifyOutput.off()
                    humidifyOn = False
                    print('humidifier off')
            elif humidTemp[0] <= humidTarget and (fanOn is True or dehumidifyOn is True):
                #It was too humid and the humidity is back down to target
                if fanOn is True:
                    fanOutput.off()
                    fanOn = False
                    print('fan off')
                elif dehumidifyOn is True:
                    dehumidifyOutput.off()
                    dehumidifyOn = False
                    print('dehumidifier off')
            elif humidTemp[0] >= humidHigh:
                #Too humid, either fan to lower, or dehumidifier
                if fanLowerHumid is True and fanOn is False:
                    fanOutput.on()
                    fanOn = True
                    print('fan on lower humid')
                elif dehumidifyPresent is True and dehumidifyOn is False:
                    humidifyOutput.on()
                    dehumidifyOn = True
                    print('dehumidifier on')
            elif humidTemp[0] < humidLow:
                #Too dry, either fan to raise or humidifier
                if fanRaiseHumid is True and fanOn is False:
                    fanOutput.on()
                    fanOn = True
                    print('fan on raise humid')
                elif humidifyPresent is True and humidifyOn is False:
                    humidifyOutput.on()
                    humidifyOn = True
                    print('humidifier on')
            
            #Control Temperature
            tempHigh = tempTarget + tempDelta
            tempLow = tempTarget - tempDelta
            if humidTemp[1] >= tempTarget and heatOn is True:
                #Turn heater off, we've reached target
                heaterOutput.off()
                heatOn = False
            elif humidTemp[1] <= tempTarget and coolerOn is True:
                #Turn cooler off, we've reached target
                coolerOutput.off()
                coolerOn = False
            #Turn Heater/Cooler on
            elif humidTemp[1] < tempLow and heatOn is False:
                #turn heat on
                heaterOutput.on()
                heatOn = True
                print('heat on')
            elif humidTemp[1] > tempHigh and coolerOn is False:
                #turn cooler on
                coolerOutput.on()
                coolerOn = True
                print('cooler on')

        #Read Buttons, handle menu stuff
        ReadButtons()
        #Now update the display
        UpdateDisplay()
except KeyboardInterrupt:
    print("KeyboardInterrupt")
processDHT.join()


<h1>Testing new DHT22 library</h1>

In [ ]:
#Libraries
import Adafruit_DHT as dht
from time import sleep
#Set DATA pin
DHT = 23
t = 10
for i in range(t):
    #Read Temp and Hum from DHT22
    h,t = dht.read_retry(dht.DHT22, DHT)
    #Print Temperature and Humidity on Shell window
    t = (t * 9 / 5) + 32
    print('Temp={0:0.1f}*F  Humidity={1:0.1f}%'.format(t,h))
    sleep(5) #Wait 5 seconds and read again

<h1>Thread Testing</h1>

In [ ]:
import threading
import os

def task1():
    print("Task 1 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 1: {}".format(os.getpid()))

def task2():
    print("Task 2 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 2: {}".format(os.getpid()))

if __name__ == "__main__":

    print("ID of process running main program: {}".format(os.getpid()))

    print("Main thread name: {}".format(threading.current_thread().name))

    t1 = threading.Thread(target=task1, name='t1')
    t2 = threading.Thread(target=task2, name='t2')

    t1.start()
    t2.start()

    t1.join()
    t2.join()